In [7]:
parameters_julia.instances_ID

LoadError: [91mUndefVarError: instances_ID not defined[39m

In [1]:
#Importing parameters
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport parameters_julia

out_dir = parameters_julia.out_dir
files_ID = parameters_julia.files_ID
month_w = parameters_julia.month_w
year = parameters_julia.year
instances_ = parameters_julia.instances_ID


instance_ = instances_[3];

In [2]:
using JSON

instance1 = instance_

#load OD pair-route incidence
odPairRoute = readstring(out_dir * "od_pair_route_incidence_" * instance1 *  files_ID * ".json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readstring(out_dir * "link_route_incidence_" * instance1 *  files_ID * ".json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readstring(out_dir *  "od_pair_label_dict_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readstring(out_dir  *"od_pair_label_dict__refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readstring(out_dir * "link_label_dict.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readstring(out_dir * "link_label_dict_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readstring(out_dir * "node_link_incidence.json");
nodeLink = JSON.parse(nodeLink);

In [3]:
include("Julia_files/load_network_uni_class.jl");

In [4]:
ta_data = load_ta_network_(out_dir, files_ID, month_w, instance1);

In [5]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [6]:
############
#Read in the demand file
file = open(out_dir * "data_traffic_assignment_uni-class/" * files_ID * "_trips_" * month_w * "_" * instance1 * ".txt")
demands = Dict()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = Int(parse(Float64,(split(line)[2])))
    elseif contains(line, ";")
        pairs = split(line, ";")
        for pair in pairs[1:end-1]
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = Int(parse(Float64,pair_vals[1])), parse(Float64,pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file);

In [7]:
demands

Dict{Any,Any} with 64 entries:
  (3, 6) => 1824.91
  (4, 4) => 0.0
  (3, 1) => 1026.72
  (4, 5) => 1152.71
  (2, 4) => 1352.56
  (6, 5) => 1438.48
  (8, 4) => 1846.1
  (1, 2) => 30.5014
  (5, 1) => 422.279
  (3, 4) => 425.322
  (8, 2) => 187.473
  (2, 3) => 6.88617
  (2, 6) => 7.80625
  (5, 4) => 598.79
  (2, 1) => 1723.62
  (2, 5) => 7.52821
  (8, 5) => 234.213
  (1, 4) => 30.372
  (2, 8) => 8.1608
  (5, 8) => 530.366
  (4, 8) => 4845.33
  (7, 8) => 1894.53
  (1, 1) => 0.0
  (7, 7) => 0.0
  (5, 7) => 503.606
  ⋮      => ⋮

In [8]:
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[odPairLabel_["$i"][1], odPairLabel_["$i"][2]]
end

In [9]:
demandsVec

56-element Array{Float64,1}:
   30.5014 
  984.779  
   30.372  
  667.846  
  670.645  
  411.881  
  436.342  
 1723.62   
    6.88617
 1352.56   
    7.52821
    7.80625
    7.72761
    ⋮      
  325.357  
  322.824  
  816.791  
  589.363  
 1894.53   
  175.974  
  187.473  
  185.953  
 1846.1    
  234.213  
  227.518  
  377.775  

In [10]:
for key=keys(odPairRoute)
    if contains(key, "56-")
        println(key)
    end
end

In [11]:
linkRoute;

In [12]:
coeffs_dict_Apr_PM_ = readstring(out_dir * "coeffs_dict_" * month_w *  "_" * instance1 * ".json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["(6, 0.5, 100.0, 1)"]

7-element Array{Any,1}:
  1.0       
  0.0055448 
  0.021304  
  0.0106823 
  0.0443759 
 -0.00108853
  0.00750175

In [13]:
using JuMP, Ipopt

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model(solver=IpoptSolver())

numLinks = length(linkLabel_)
numRoute = length(linkRoute)
numOD = length(demandsVec)

@variable(m, linkFlow[1:numLinks])
@variable(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @constraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()



for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @constraint(m, pathFlowLinkSum[a] == linkFlow[a])
end



for j=1:numRoute
    @NLconstraint(m, pathFlow[j] >= 0)
end

#@NLexpression(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@NLexpression(m, f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 , a = 1:numLinks})



@NLobjective(m, Min, f)
#print(m) 

solve(m);

56-82


In [14]:
getvalue(linkFlow);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      294
Number of nonzeros in inequality constraint Jacobian.:      192
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:      216
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [15]:
getobjectivevalue(m)

16746.64125997778

In [16]:
flows = Dict()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getvalue(linkFlow)[i]
end

flows

Dict{Any,Any} with 24 entries:
  (3, 6) => 2503.36
  (2, 3) => 1886.86
  (5, 6) => 3823.52
  (5, 4) => 1708.89
  (2, 1) => 2620.48
  (6, 3) => 1521.97
  (3, 1) => 2411.31
  (1, 3) => 1322.74
  (4, 5) => 3775.97
  (4, 8) => 4845.33
  (7, 5) => 2857.77
  (8, 7) => 1388.91
  (3, 2) => 3831.03
  (7, 8) => 4668.87
  (2, 4) => 1382.93
  (7, 6) => 816.88
  (6, 5) => 1800.43
  (6, 7) => 3360.23
  (8, 4) => 1846.1
  (1, 2) => 1909.62
  (5, 3) => 3632.31
  (4, 2) => 2247.91
  (3, 5) => 5368.14
  (5, 7) => 3926.97

In [17]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
#using Conda
#using Pandas

@pyimport matplotlib.pyplot as plt
@pyimport numpy as np
@pyimport json
@pyimport os
@pyimport pickle
@pyimport pandas as pd
@pyimport collections


#@pyimport pandas
#@pyimport parameters_julia
#@pyimport utils_julia
#@pyimport GLS_julia

LoadError: [91m@pyimport: json already defined[39m

In [18]:
@pyimport parameters_julia
@pyimport utils_julia
@pyimport GLS_julia

In [19]:
flow_user  = GLS_julia.x_;

In [20]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
                    free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
                    free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
                    free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
                    free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [21]:
PoA_dict = Dict();
tapSocialFlowDicDict = Dict();
tapSocialFlowVecDict = Dict();

week_day_Apr_list = 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30
for i = 1:length(week_day_Apr_list)
    
    println(socialObj(flow_user[:, i])/getobjectivevalue(m))
end

In [22]:
getobjectivevalue(m)

16746.64125997778

0.9184266650537098
1.1633710310020742
1.118226345945591
1.326664874201384
1.0002566696323687
0.8808867584754252
0.9517331343277949
1.0913553927729396
1.4073545905393279
1.1281934710459962
0.7914033787966687
1.08650933580465
1.251226810142686
1.1984619195126756
1.068986251102524
0.8907034607153301
1.0217381735693019
1.084734504517406
1.2448734751355035
1.137044784955886
1.0200957905606225
